In [ ]:
from pathlib import Path
import pandas as pd

data_dir = Path("/home/sean/Projects/streambt/full_trade_opportunities.parquet/")
full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')
)

In [ ]:
full_df

In [ ]:
import duckdb as db
win_loss_ratio_ordered = db.sql(
"""
    select 
        *
        , "trade won"/"trade loss" as win_loss_ratio
        , dense_rank() over (partition by "year(Date)" order by win_loss_ratio desc) as drnk
    from full_df
"""
)
ticker_with_good_year_stat = db.sql(
"""
    select * 
    from win_loss_ratio_ordered
    where drnk < 10 and "average trade return" >= 1.05 and win_loss_ratio > 2 ---and "year(Date)" <= 2020
    --order by "year(Date)", win_loss_ratio desc
"""
) 
# verify tickers with good occurrance count
occ_of_ticker = db.sql(
"""
    select count(*) as occurrance, Ticker
    from ticker_with_good_year_stat
    group by Ticker
    --order by occurrance desc
"""
)
# subsequent binning
binning = db.sql(
"""
select count(*) as val, occurrance
from occ_of_ticker
group by occurrance
order by occurrance asc
"""
).df()

filter_for_consistently_good_ticker = db.sql("""select * from occ_of_ticker where occurrance >= 10 order by occurrance desc""")
print("cnt of consistently good tickers\n", db.sql("select count(*) from filter_for_consistently_good_ticker"))

# verify the yoy performance of these stocks
filter_performance_verification = db.sql(
"""
select *
from win_loss_ratio_ordered t
inner join filter_for_consistently_good_ticker f on t.Ticker = f.Ticker 
order by "year(Date)", win_loss_ratio desc
"""
)

import plotly.express as px
ax = px.bar(binning,x='occurrance',y='val')
ax.show()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(filter_for_consistently_good_ticker.df())
    display(filter_performance_verification.df())
    